# Create input data
This notebook does all processing to turn input-data.xlsx, all other input data into a format to put into insertions.xlsx

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# read input data
input_data_filename = '../../../Paper 3 - Charging profiles/input_data/input-data.xlsx'
# Read all sheets into a dictionary
input_data_dict = pd.read_excel(input_data_filename, sheet_name=None)

input_params = input_data_dict['inputs'].set_index('parameter')['value'].to_dict()

truck_specs = input_data_dict['truck_specs']

In [3]:
# join eRoads bins to get AADTT and length
eRoads_data = (input_data_dict['MTO iCorridor AADTT']
               .dropna(axis='columns', how='any')
               .rename(columns = {'LENGTH': 'eRoads_length_km'}))
eRoads_data_summary = (eRoads_data
                       .groupby('Bin').agg({'eRoads_length_km':'sum', 'AADTT19xLength':'sum'})
                       .assign(eRoads_length_weighted_aadtt19 = lambda x: x['AADTT19xLength'] / x['eRoads_length_km']).reset_index()
                       .drop(columns='AADTT19xLength')
)
eRoads_data_summary

truck_specs = truck_specs.merge(eRoads_data_summary, how='left')

# Define truck specifications
- From input-data.xlsx
- Truck data from Islam et al (2023) Autonomie
- Prepare truck specs, weights, energy consumption, costs with Autonomie Islam 2023 for each long-haul truck config
- Filter to Purpose=Longhaul only
- Output: data for each technology to construct tables 'MaxAnnualCapacityFactor', 'CostInvest'
- Efficiency doesnt need to be recalculated, just reuse existing spreadsheet, but separate into non long-haul and long-haul
- 'CostVariable' just reuse existing spreadsheet

To be appended in sheet 'Autonomie 2023 MHDV', with column names:
- Vehicle Manufacturing Cost ($ 2023USD)

### Analyze Autonomie data

In [5]:
# Read the Excel file and select the relevant sheet
file_path = 'Autonomie HDT.xlsx'
autonomie_df = pd.read_excel(file_path, sheet_name='class8_longhaul_regional')

# Convert mpgde to kWh/km
# 1 gallon diesel = 37.656 kWh, 1 mile = 1.60934 km
# 128,488 Btu/gal Low Sulfur Diesel https://afdc.energy.gov/fuels/properties
autonomie_df['kWh_per_km'] = 37.656 / (autonomie_df['mpgde'] * 1.60934)

autonomie_df = autonomie_df[autonomie_df['Purpose'] == "Longhaul"]

In [6]:
# Function to compute scaled vehicle cost and weight based on desired range
def compute_scaled_attributes(df, Powertrain, Year, Case, Purpose, RegCode, desired_range_km, 
                              eRoads_equipment_kg = 0, eRoads_equipment_cost = 0,
                              **kwargs):
    subset = df[(df['Powertrain'] == Powertrain) &
                (df['Year'] == Year) &
                (df['Case'] == Case) &
                (df['Purpose'] == Purpose) &
                (df['RegCode'] == RegCode)]

    if subset.empty:
        raise ValueError("No matching data found for the provided filters.")
    if subset.shape[0] > 1:
        raise ValueError("Multiple matches found for filter conditions.")

    row = subset.iloc[0].to_dict()
    desired_range_miles = desired_range_km / 1.60934
    scale_factor = desired_range_miles / row['Range']

    Pack_Energy_kWh = row['Pack_Energy_kWh']  # Default value
    H2_Mass_kg = row['H2_Mass_kg']  # Default value

    if Powertrain == 'BEV':
        scaled_battery_kg = row['Battery_kg'] * scale_factor
        scaled_battery_cost = row['Battery_Cost'] * scale_factor
        
        weight_minus_battery = row['CurbWeight'] - row['Battery_kg']
        cost_minus_battery = row['VehicleCost'] - row['Battery_Cost']

        vehicle_weight = weight_minus_battery + scaled_battery_kg + eRoads_equipment_kg
        vehicle_cost = cost_minus_battery + scaled_battery_cost + eRoads_equipment_cost

        Pack_Energy_kWh *= scale_factor

    elif Powertrain == 'FCEV':
        scaled_h2_tank_kg = row['H2_Tank_kg'] * scale_factor
        scaled_h2_storage_cost = row['H2_Storage_Cost'] * scale_factor

        weight_minus_tank = row['CurbWeight'] - row['H2_Tank_kg']
        cost_minus_tank = row['VehicleCost'] - row['H2_Storage_Cost']
        
        vehicle_weight = weight_minus_tank + scaled_h2_tank_kg
        vehicle_cost = cost_minus_tank + scaled_h2_storage_cost

        H2_Mass_kg *= scale_factor

    else:
        raise ValueError("Powertrain must be 'BEV' or 'FCEV'.")
    return {
            'CurbWeight': vehicle_weight,
            'VehicleCost': vehicle_cost,
            'PayloadCapacity': row['GVWR'] - vehicle_weight,
            'kWh_per_km': row['kWh_per_km'],
            'Pack_Energy_kWh': Pack_Energy_kWh, 
            'H2_Mass_kg': H2_Mass_kg,
        }

In [7]:
compute_scaled_attributes(autonomie_df, **autonomie_df.iloc[0].to_dict(), desired_range_km=1000)

{'CurbWeight': 20522.365678874936,
 'VehicleCost': 382209.7582772525,
 'PayloadCapacity': 15764.994321125065,
 'kWh_per_km': 1.4867290445304828,
 'Pack_Energy_kWh': 1473.4512285386477,
 'H2_Mass_kg': 0.0}

### Get average payload from CFAF

In [8]:
region = 'ON'
long_haul_threshold = 600

mean_weight_abvthreshold = (pd.read_csv('cfaf_province_longhaul_threshold_stats.csv').query(f'region == "{region}" and long_haul_threshold=={long_haul_threshold}')
                            ['mean_weight_abvthreshold'].iloc[0])
print(f"Mean weight above threshold for {region} with long haul threshold {long_haul_threshold} km: {round(mean_weight_abvthreshold)} kg")

Mean weight above threshold for ON with long haul threshold 600 km: 8181 kg


### Get energy consumption, battery, H2 size

In [9]:
# expand 'Year', add 'Case'
truck_specs = (truck_specs.merge(autonomie_df[['Year']].drop_duplicates(), how='cross')
            .assign(Case=input_params['autonomie_tech_case'])
            )
truck_specs = truck_specs.assign(Purpose = "Longhaul", RegCode = "Sleeper")

# compute scaled attributes
truck_specs = pd.concat([truck_specs, 
                      truck_specs.apply(lambda x: compute_scaled_attributes(df=autonomie_df, **x), axis='columns', result_type='expand')], 
                      axis='columns')

# Add payload capacity penalty relative to mean_weight_abvthreshold
truck_specs['PayloadCapacityPenalty'] = np.clip(mean_weight_abvthreshold / truck_specs['PayloadCapacity'], a_min=1, a_max=999)

truck_specs#.drop(columns='notes').query('Year==2030')

,tech_veh,tech_supply,Bin,Powertrain,desired_range_km,eRoads_equipment_kg,eRoads_equipment_cost,charger_standard_kW,charger_fast_kW,notes,...,Case,Purpose,RegCode,CurbWeight,VehicleCost,PayloadCapacity,kWh_per_km,Pack_Energy_kWh,H2_Mass_kg,PayloadCapacityPenalty
0,T_HDV_T_LH_BEV_LRGBATT_N,T_HDV_T_LH_CHRG_LRGBATT,NaN,BEV,1206.75,0,0,350.0,350.0,desired range: 1.5x of Autonomie range (500 mi...,...,Low,Longhaul,Sleeper,26055.151419,817170.316581,10232.208581,1.774878,2037.391664,0.000000,1.0
1,T_HDV_T_LH_BEV_LRGBATT_N,T_HDV_T_LH_CHRG_LRGBATT,NaN,BEV,1206.75,0,0,350.0,350.0,desired range: 1.5x of Autonomie range (500 mi...,...,Low,Longhaul,Sleeper,23886.756398,636636.174090,12400.603602,1.596685,1906.901800,0.000000,1.0
2,T_HDV_T_LH_BEV_LRGBATT_N,T_HDV_T_LH_CHRG_LRGBATT,NaN,BEV,1206.75,0,0,350.0,350.0,desired range: 1.5x of Autonomie range (500 mi...,...,Low,Longhaul,Sleeper,21336.847100,454936.731562,14950.512900,1.440349,1769.152158,0.000000,1.0
3,T_HDV_T_LH_BEV_LRGBATT_N,T_HDV_T_LH_CHRG_LRGBATT,NaN,BEV,1206.75,0,0,350.0,350.0,desired range: 1.5x of Autonomie range (500 mi...,...,Low,Longhaul,Sleeper,19289.266037,362307.990210,16998.093963,1.288763,1641.826813,0.000000,1.0
4,T_HDV_T_LH_BEV_LRGBATT_N,T_HDV_T_LH_CHRG_LRGBATT,NaN,BEV,1206.75,0,0,350.0,350.0,desired range: 1.5x of Autonomie range (500 mi...,...,Low,Longhaul,Sleeper,16530.916384,236516.208615,19756.443616,1.128199,1540.189915,0.000000,1.0
5,T_HDV_T_LH_BEV_FSTCHRG_N,T_HDV_T_LH_CHRG_FSTCHRG,NaN,BEV,804.50,0,0,350.0,3000.0,desired range: same as Autonomie range (500 mi...,...,Low,Longhaul,Sleeper,21353.478350,579474.544387,14933.881650,1.774878,1358.261109,0.000000,1.0
6,T_HDV_T_LH_BEV_FSTCHRG_N,T_HDV_T_LH_CHRG_FSTCHRG,NaN,BEV,804.50,0,0,350.0,3000.0,desired range: same as Autonomie range (500 mi...,...,Low,Longhaul,Sleeper,19800.538255,458658.782726,16486.821745,1.596685,1271.267867,0.000000,1.0
7,T_HDV_T_LH_BEV_FSTCHRG_N,T_HDV_T_LH_CHRG_FSTCHRG,NaN,BEV,804.50,0,0,350.0,3000.0,desired range: same as Autonomie range (500 mi...,...,Low,Longhaul,Sleeper,18019.686804,336993.154375,18267.673196,1.440349,1179.434772,0.000000,1.0
8,T_HDV_T_LH_BEV_FSTCHRG_N,T_HDV_T_LH_CHRG_FSTCHRG,NaN,BEV,804.50,0,0,350.0,3000.0,desired range: same as Autonomie range (500 mi...,...,Low,Longhaul,Sleeper,16639.299952,274743.993473,19648.060048,1.288763,1094.551209,0.000000,1.0
9,T_HDV_T_LH_BEV_FSTCHRG_N,T_HDV_T_LH_CHRG_FSTCHRG,NaN,BEV,804.50,0,0,350.0,3000.0,desired range: same as Autonomie range (500 mi...,...,Low,Longhaul,Sleeper,14743.535495,190310.472410,21543.824505,1.128199,1026.793277,0.000000,1.0


### Calculate CapacityToActivity for infra

In [15]:
def calc_CapacityToActvity_infra(tech_supply, charger_standard_kW, charger_fast_kW, eRoads_length_weighted_aadtt19, kWh_per_km, **kwargs):
    if (tech_supply == 'T_HDV_T_LH_CHRG_LRGBATT'): # or (tech_supply == 'T_H2_HDV_T_LH_REFUEL'):
        return charger_standard_kW*60*60*24*365/1e12    # 60 seconds * 60 minutes * 24 hours * 365 days / 1e12 to convert to PJ
    elif tech_supply == 'T_HDV_T_LH_CHRG_FSTCHRG':
        return charger_fast_kW*60*60*24*365/1e12    # 60 seconds * 60 minutes * 24 hours * 365 days / 1e12 to convert to PJ
        # skip this
        # return ((charger_standard_kW*input_params['overnight_charging_share']) + (charger_fast_kW*(1-input_params['overnight_charging_share'])))*60*60*24*365/1e12
    elif 'CHRG_ERDS' in tech_supply:
        return eRoads_length_weighted_aadtt19 * 365 * kWh_per_km * 3.6 / 1e9  # convert to PJ
    else:
        return np.nan  # Default case if no conditions are met

In [16]:
# apply the function to calculate CapacityToActivity_infra and keep original columns
truck_specs['CapacityToActivity_infra'] = (truck_specs.apply(lambda x: calc_CapacityToActvity_infra(**x), axis='columns', result_type='expand'))

# Output data for insertions.xlsx

In [17]:
# output costs for CostInvest
(truck_specs.query('Case=="Low"').pivot(index='tech_veh', columns='Year', values='VehicleCost')
 .reset_index().style.hide())

tech_veh,2023,2025,2030,2035,2050
T_HDV_T_LH_BEV_ERDS1_N,349278.772194,288181.391363,226549.577187,194679.996737,151604.736205
T_HDV_T_LH_BEV_ERDS2_N,349278.772194,288181.391363,226549.577187,194679.996737,151604.736205
T_HDV_T_LH_BEV_ERDS3_N,349278.772194,288181.391363,226549.577187,194679.996737,151604.736205
T_HDV_T_LH_BEV_ERDS4_N,349278.772194,288181.391363,226549.577187,194679.996737,151604.736205
T_HDV_T_LH_BEV_ERDS5_N,349278.772194,288181.391363,226549.577187,194679.996737,151604.736205
T_HDV_T_LH_BEV_FSTCHRG_N,579474.544387,458658.782726,336993.154375,274743.993473,190310.472410
T_HDV_T_LH_BEV_LRGBATT_N,817170.316581,636636.174090,454936.731562,362307.990210,236516.208615
T_HDV_T_LH_FCEV_N,326090.295008,276845.430267,240462.180325,217235.423343,180532.654245


In [18]:
# get payload penalty factors
(truck_specs.query('Case=="Low"').pivot(index='tech_veh', columns='Year', values='PayloadCapacityPenalty')
 .reset_index().style.hide())

tech_veh,2023,2025,2030,2035,2050
T_HDV_T_LH_BEV_ERDS1_N,1.000000,1.000000,1.000000,1.000000,1.000000
T_HDV_T_LH_BEV_ERDS2_N,1.000000,1.000000,1.000000,1.000000,1.000000
T_HDV_T_LH_BEV_ERDS3_N,1.000000,1.000000,1.000000,1.000000,1.000000
T_HDV_T_LH_BEV_ERDS4_N,1.000000,1.000000,1.000000,1.000000,1.000000
T_HDV_T_LH_BEV_ERDS5_N,1.000000,1.000000,1.000000,1.000000,1.000000
T_HDV_T_LH_BEV_FSTCHRG_N,1.000000,1.000000,1.000000,1.000000,1.000000
T_HDV_T_LH_BEV_LRGBATT_N,1.000000,1.000000,1.000000,1.000000,1.000000
T_HDV_T_LH_FCEV_N,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
# CapacityToActivity for infrastructure
(truck_specs.groupby('tech_supply')['CapacityToActivity_infra']
 .max()    # take the maximum value observed
 .reset_index()
 .style.hide())

tech_supply,CapacityToActivity_infra
T_H2_HDV_T_LH_REFUEL,nan
T_HDV_T_LH_CHRG_ERDS1,0.036925
T_HDV_T_LH_CHRG_ERDS2,0.013693
T_HDV_T_LH_CHRG_ERDS3,0.003662
T_HDV_T_LH_CHRG_ERDS4,0.001391
T_HDV_T_LH_CHRG_ERDS5,0.000204
T_HDV_T_LH_CHRG_FSTCHRG,0.094608
T_HDV_T_LH_CHRG_LRGBATT,0.011038


In [30]:
# MaxCapacity for eRoads
(truck_specs[['tech_supply', 'eRoads_length_km']].drop_duplicates().dropna()
 # expand over all periods
 #.merge(pd.DataFrame({'period': [2021, 2025, 2030, 2035, 2040, 2045, 2050]}), how='cross')
 .style.format(precision=2).hide()
)

tech_supply,eRoads_length_km
T_HDV_T_LH_CHRG_ERDS1,838.85
T_HDV_T_LH_CHRG_ERDS2,838.87
T_HDV_T_LH_CHRG_ERDS3,2499.47
T_HDV_T_LH_CHRG_ERDS4,4176.97
T_HDV_T_LH_CHRG_ERDS5,8426.22


# Create profiles for CapacityFactorTech

In [3]:
def create_CapacityFactorTech_single_day(profile_name, input_data_dict, season_list):
    """
    Create a DataFrame with technology-specific capacity factors for each season.
    Only for a single day profile which is repeated across all seasons in season_list.
    Returns a DataFrame to be inserted into CapacityFactorTech.
    """
    hourly_profile = input_data_dict['sample_profiles'].set_index('tod')[profile_name]
    hourly_profile.name = 'factor'

    df = (hourly_profile.reset_index()
        .merge(pd.DataFrame({'season': season_list}), how='cross')
        .merge(input_data_dict['truck_specs']['tech_supply'].rename('tech'), how='cross')
        .assign(region='ON', cf_tech_notes=profile_name)
        )
    return df

In [4]:
season_list = ["D222", "D007", "D185", "D217", "D295", "D248", "D332", "D214", "D160", "D067", "D290", "D054"]
create_CapacityFactorTech_single_day('flat', input_data_dict, season_list)

,tod,factor,season,tech,region,cf_tech_notes
0,H01,1,D222,T_HDV_T_LH_CHRG_LRGBATT,ON,flat
1,H01,1,D222,T_HDV_T_LH_CHRG_FSTCHRG,ON,flat
2,H01,1,D222,T_HDV_T_LH_CHRG_ERDS1,ON,flat
3,H01,1,D222,T_HDV_T_LH_CHRG_ERDS2,ON,flat
4,H01,1,D222,T_HDV_T_LH_CHRG_ERDS3,ON,flat
...,...,...,...,...,...,...
2299,H24,1,D054,T_HDV_T_LH_CHRG_ERDS2,ON,flat
2300,H24,1,D054,T_HDV_T_LH_CHRG_ERDS3,ON,flat
2301,H24,1,D054,T_HDV_T_LH_CHRG_ERDS4,ON,flat
2302,H24,1,D054,T_HDV_T_LH_CHRG_ERDS5,ON,flat


In [6]:
from lihwei_db_edits import insert_profiles_CapacityFactorTech

In [8]:
insert_profiles_CapacityFactorTech('canoe_on_12d_baseline_newtech.sqlite', 
                                   profiles_df=create_CapacityFactorTech_single_day('flat', input_data_dict, season_list),
                                   new_db_path='canoe_on_12d_baseline_newtech_flat.sqlite')

Copying database from canoe_on_12d_baseline_newtech.sqlite to canoe_on_12d_baseline_newtech_flat.sqlite for insertion of profiles.
